In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/commonlit-evaluate-student-summaries/sample_submission.csv
/kaggle/input/commonlit-evaluate-student-summaries/prompts_train.csv
/kaggle/input/commonlit-evaluate-student-summaries/summaries_test.csv
/kaggle/input/commonlit-evaluate-student-summaries/summaries_train.csv
/kaggle/input/commonlit-evaluate-student-summaries/prompts_test.csv


In [2]:
def print_tuple(obs):
    print("\033[1mprompt_question: \033[0m", obs.prompt_question)
    print()
    print("\033[1mprompt_title: \033[0m", obs.prompt_title)
    print()
    print("\033[1mprompt_text: \033[0m")
    print(obs.prompt_text)
    print()
    print("\033[1msummary: \033[0m")
    print(obs.text)
    print()
    print("\033[1mWording: \033[0m")
    print(obs.wording)
    print()
    print("\033[1mContent: \033[0m")
    print(obs.content)
    

In [3]:
#Filepath
FILE_PATH='/kaggle/input/commonlit-evaluate-student-summaries'
#train set
pmt_train=pd.read_csv(os.path.join(FILE_PATH,'prompts_train.csv'))
sum_train=pd.read_csv(os.path.join(FILE_PATH,'summaries_train.csv'))
df_train=pmt_train.merge(sum_train,how='inner',on='prompt_id')
#we actually dont want student_id or prompt_id for training hence dropping those.
df_train.drop(['student_id','prompt_id'],axis=1,inplace=True)

In [4]:
#Checking whether we got a balanced dataset
print(df_train['prompt_title'].value_counts())
#shuffling the dataset
df_train=df_train.sample(frac=0.1)

On Tragedy                   2057
Egyptian Social Structure    2009
Excerpt from The Jungle      1996
The Third Wave               1103
Name: prompt_title, dtype: int64


In [5]:
import random
index=random.randint(0,len(df_train)-3)
for x in range(index,index+5):
    print_tuple(df_train.iloc[x])

prompt_question:  Summarize the various ways the factory would use or cover up spoiled meat. Cite evidence in your answer.

prompt_title:  Excerpt from The Jungle

prompt_text: 
With one member trimming beef in a cannery, and another working in a sausage factory, the family had a first-hand knowledge of the great majority of Packingtown swindles. For it was the custom, as they found, whenever meat was so spoiled that it could not be used for anything else, either to can it or else to chop it up into sausage. With what had been told them by Jonas, who had worked in the pickle rooms, they could now study the whole of the spoiled-meat industry on the inside, and read a new and grim meaning into that old Packingtown jest—that they use everything of the pig except the squeal. 
Jonas had told them how the meat that was taken out of pickle would often be found sour, and how they would rub it up with soda to take away the smell, and sell it to be eaten on free-lunch counters; also of all the m